# ReAct with Bedrock: using tools

In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from IPython.display import display, Markdown
import os
import wandb



In [ ]:
pip show langchain

## Setting up LLM

In [2]:
# OpenAI or Bedrock
#llm = OpenAI(temperature=0)

from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
import os
import sys
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from myutils import bedrock, print_ww

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['AWS_PROFILE'] = 'sengledo+bedrock-benelux-Admin'
boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))


#model_kwargs={'max_tokens_to_sample':200}
model_kwargs={
        'temperature': 0.0, 'maxTokenCount': 4096
    }

llm = Bedrock(model_id="amazon.titan-tg1-large", client=boto3_bedrock)
#llm = Bedrock(model_id="anthropic.claude-v1", client=boto3_bedrock)

Create new client
  Using region: us-east-1
  Using profile: sengledo+bedrock-benelux-Admin
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


### AgentType.ZERO_SHOT_REACT_DESCRIPTION
https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html?highlight=agenttype#langchain.agents.agent_types.AgentType

ReACT using wikipedia and serapi tools

In [ ]:
#tools = load_tools(["serpapi", "wikipedia", "llm-math"], llm=llm)
tools = load_tools(["wikipedia"], llm=llm)
agent = initialize_agent(tools, llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         handle_parsing_errors=True)

#agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")
q = "The first woman to receive a Ph.D. in computer science was Dr. Barbara Liskov, who earned her degree from Stanford University in 1968."
#q="Who is the first woman to receive a Ph.D. in computer science?"
#q = "Jair Bolsonaro was the first man to walk over the moon in 1967."
answer = agent.run(q)
print(q)
#template = """Is this observation correct: {observation}"""
#answer = agent.run(template.format(observation=q))
#print(template.format(observation=q))
display(Markdown(answer))

# Agents + CoT + Bedrock

### AgentType.CONVERSATIONAL_REACT_DESCRIPTION

In [ ]:
#https://amzn-aws.slack.com/archives/C05EBNPFEPN/p1692273353672639?thread_ts=1692270494.852529&cid=C05EBNPFEPN
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")
tools = load_tools(["serpapi","wikipedia"], llm=llm)

agent_chain = initialize_agent(tools, llm,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True,
                               memory=memory
                               ,
                               agent_kwargs={'prefix': '''
Assistant is a large language model trained by Amazon.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
TOOLS:
------
Assistant has access to the following tools:'''})
#agent_chain.run(input="hi, i am Joseba")
#agent_chain.run(input="What's the first letter in my name?")
agent_chain.run(input= "Is this statement correct? The first woman to receive a Ph.D. in computer science was Dr. Barbara Liskov, who earned her degree from Stanford University in 1968.")

### AgentType.ZERO_SHOT_REACT_DESCRIPTION

In [3]:
#https://amzn-aws.slack.com/archives/C05EBNPFEPN/p1692273353672639?thread_ts=1692270494.852529&cid=C05EBNPFEPN
from langchain.memory import ConversationBufferMemory
tools = load_tools(["serpapi","wikipedia"], llm=llm)

agent = initialize_agent(tools, llm,
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True,
                               handle_parsing_errors=True,
                               )
input = """Here is a statement:
    {statement}
    Is this statement is correct? You can use tools to find information if needed.
    The final response is FALSE if the statement is FALSE. Otherwise, TRUE."""

q= "The first woman to receive a Ph.D. in computer science was Dr. Barbara Liskov, who earned her degree from Stanford University in 1968."
agent.run(input.format(statement=q))




> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action::  I need to verify if the first woman to receive a Ph.D. in computer science was Dr. Barbara Liskov
Action: I will use Wikipedia to search for "first woman to receive a Ph.D. in computer science"
Action Input: first woman to receive a Ph.D. in computer science

Thought: I now know the final answer
Final Answer: FALSE
Observation: Invalid or incomplete response
Thought: I now know the final answer
Final Answer: FALSE

> Finished chain.


'FALSE'

## Setting up wandb


In [ ]:
WANDB_NOTEBOOK_NAME='ReAct'
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
wandb.login(key=os.environ['WANDB_API_KEY'])

run = wandb.init(
    project="fakenewsevaluation", 
    job_type="generation")

# Define W&B Table to store generations
columns = ["model", "question", "answer", "prompt"]
table = wandb.Table(columns=columns)

In [ ]:
# Using OpenAI

#llm = OpenAI(temperature=0)
#agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
#agent.run("How many people live in canada as of 2023?")